In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
import pandas as pd

def statement(r):
    p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
    check = re.findall('var originalData = (.*?);\r\n\r\n\r', r.text)
    data = json.loads(p.findall(r.text)[0])
    headers = list(data[0].keys())
    headers.remove('popup_icon')
    result = []
    for row in data:
        soup = bs(row['field_name'])
        field_name = soup.select_one('a, span').text
        fields = list(row.values())[2:]
        fields.insert(0, field_name)
        result.append(fields)
    pd.option_context('display.max_rows', None, 'display.max_columns', None)
    return pd.DataFrame(result, columns = headers)
    

Income_statment = statement(requests.get('https://www.macrotrends.net/stocks/charts/AAPL/apple/income-statement?freq=Q'))
Balance_sheet = statement(requests.get('https://www.macrotrends.net/stocks/charts/AAPL/apple/balance-sheet?freq=Q'))
Cash_flow = statement(requests.get('https://www.macrotrends.net/stocks/charts/AAPL/apple/cash-flow-statement?freq=Q'))

frames = [Income_statment, Balance_sheet, Cash_flow]
df = pd.concat(frames)
df.rename(columns={'field_name':'Date'}, 
                 inplace=True)
Finance_statment = df.transpose()
#print(Finance_statment)
#Finance_statment['field_name'] =pd.to_datetime(Finance_statment.Date)
#Finance_statment.sort('field_name')
Finance_statment.to_csv('Financial_statement.csv', index = True, header=False)
fs = pd.read_csv('Financial_statement.csv')
fs['Date'] = pd.to_datetime(fs.Date)
fs.sort_values(by=['Date'], inplace=True, ascending=True)
#print(fs.head())
fs.to_csv('Financial_statement.csv', index = False, header=True)